In [274]:
import os
import json
import requests
import pandas as pd
from mysql import connector
from dotenv import load_dotenv
import numpy as np
from datetime import datetime


In [275]:
coins = ['bitcoin', 'solana']

load_dotenv()

url_base = "https://api.coingecko.com/api/v3/coins/"
days = 90
params = {
    "vs_currency": "usd",
    "days": days
}

print(params)

{'vs_currency': 'usd', 'days': 90}


# EXTRACT

In [276]:
payloads = []


for idx, coin in enumerate(coins):
    url = f"{url_base}{coin}/market_chart"
    response = requests.get(url, params=params)
    payloads.append(response.json())
    
formatted_response = json.dumps(payloads, indent=4)
print(formatted_response)

[
    {
        "prices": [
            [
                1750554199810,
                102949.09073657315
            ],
            [
                1750557805721,
                102511.70596202089
            ],
            [
                1750561413094,
                102372.11654090365
            ],
            [
                1750565005307,
                102328.93433953862
            ],
            [
                1750568609501,
                102598.19431204248
            ],
            [
                1750572204045,
                102751.21939870145
            ],
            [
                1750575819832,
                102750.98957518884
            ],
            [
                1750579395233,
                102431.3057604344
            ],
            [
                1750582991990,
                102653.05717196758
            ],
            [
                1750586612754,
                102236.88032084289
            ],
            [
         

# TRANSFORM

In [277]:
market_history_df1 = pd.DataFrame(payloads)
market_history_df1

,prices,market_caps,total_volumes
0,"[[1750554199810, 102949.09073657315], [1750557...","[[1750554199810, 2047973961516.251], [17505578...","[[1750554199810, 25534220303.202923], [1750557..."
1,"[[1750554199301, 136.88048778730024], [1750557...","[[1750554199301, 72729456819.55728], [17505578...","[[1750554199301, 4291376810.0359163], [1750557..."


In [278]:
def prepare_market_history_records(coin_id, market_chart_data, sample_size=(3 * days)):
    rows = []
    total_records = len(market_chart_data['prices'])
    indices = np.linspace(0, total_records - 1, sample_size)
    indices = np.floor(indices).astype(int)

    print(indices)

    for i in indices:
        timestamp_ms = market_chart_data['prices'][i][0]
        timestamp = datetime.fromtimestamp(timestamp_ms / 1000)

        price = round(market_chart_data['prices'][i][1], 5)
        market_cap = round(market_chart_data['market_caps'][i][1], 5)
        volume = round(market_chart_data['total_volumes'][i][1], 5)

        row = {
            'coin_id': coin_id,
            'timestamp': timestamp,
            'price': price,
            'market_cap': market_cap,
            'volume': volume
        }
        rows.append(row)

    return rows

fixed_payload = []
for idx, coin in enumerate(coins):
    fixed_payload.extend(prepare_market_history_records(coin, payloads[idx]))

market_history_df = pd.DataFrame(fixed_payload)
market_history_df


[   0    8   16   24   32   40   48   56   64   72   80   88   96  104
  112  120  128  136  144  152  160  168  176  184  192  200  208  216
  224  232  240  248  256  264  272  280  288  296  304  313  321  329
  337  345  353  361  369  377  385  393  401  409  417  425  433  441
  449  457  465  473  481  489  497  505  513  521  529  537  545  553
  561  569  577  585  593  601  609  618  626  634  642  650  658  666
  674  682  690  698  706  714  722  730  738  746  754  762  770  778
  786  794  802  810  818  826  834  842  850  858  866  874  882  890
  898  906  914  922  931  939  947  955  963  971  979  987  995 1003
 1011 1019 1027 1035 1043 1051 1059 1067 1075 1083 1091 1099 1107 1115
 1123 1131 1139 1147 1155 1163 1171 1179 1187 1195 1203 1211 1219 1227
 1236 1244 1252 1260 1268 1276 1284 1292 1300 1308 1316 1324 1332 1340
 1348 1356 1364 1372 1380 1388 1396 1404 1412 1420 1428 1436 1444 1452
 1460 1468 1476 1484 1492 1500 1508 1516 1524 1532 1540 1549 1557 1565
 1573 

,coin_id,timestamp,price,market_cap,volume
0,bitcoin,2025-06-21 21:03:19.810,102949.09074,2.047974e+12,2.553422e+10
1,bitcoin,2025-06-22 05:03:11.990,102653.05717,2.039764e+12,3.072199e+10
2,bitcoin,2025-06-22 13:03:44.309,99518.50794,1.977044e+12,4.965522e+10
3,bitcoin,2025-06-22 21:03:32.448,101095.57550,2.012897e+12,4.963503e+10
4,bitcoin,2025-06-23 05:03:42.449,101885.85927,2.025801e+12,4.930648e+10
...,...,...,...,...,...
535,solana,2025-09-18 12:00:52.654,248.44757,1.349469e+11,1.210724e+10
536,solana,2025-09-18 20:01:45.733,247.59944,1.344311e+11,9.510189e+09
537,solana,2025-09-19 04:02:09.693,244.11601,1.326490e+11,9.091177e+09
538,solana,2025-09-19 12:01:36.829,237.77728,1.291426e+11,9.165780e+09


In [279]:
MYSQL_USER = os.getenv("MYSQL_USER")
MYSQL_PASSWORD = os.getenv("MYSQL_PASSWORD")
MYSQL_HOST = os.getenv("MYSQL_HOST")
MYSQL_PORT = os.getenv("MYSQL_PORT")
MYSQL_DB = os.getenv("MYSQL_DATABASE")

In [280]:
db_conn = connector.connect(
    host=MYSQL_HOST,
    user=MYSQL_USER,
    password=MYSQL_PASSWORD,
    port=MYSQL_PORT,
    database=MYSQL_DB,
    connection_timeout=10,
    autocommit=False,
    raise_on_warnings=True
)   

db_cur = db_conn.cursor()
print(f"[SUCCESS] Connected to MySQL db {MYSQL_HOST}:{MYSQL_PORT}/{MYSQL_DB} as user {MYSQL_USER}")

[SUCCESS] Connected to MySQL db localhost:3306/cryptodb as user root


In [281]:
sql_table = "market_history"
db_cur.execute(f"SHOW TABLES LIKE '{sql_table}'")

if db_cur.fetchone() is None:
    raise SystemExit(f"[ERROR] Table '{sql_table}' does not exist in database '{MYSQL_DB}'")
else:
    print(f"[SUCCESS] Table '{sql_table}' exists in database '{MYSQL_DB}'")

[SUCCESS] Table 'market_history' exists in database 'cryptodb'


In [282]:
INSERT_SQL = """
INSERT INTO market_history (
    coin_id, timestamp, price, market_cap, volume
) VALUES (
    %s, %s, %s, %s, %s
)
"""

In [283]:
market_history_list = market_history_df.values.tolist()

In [284]:
try:
    db_cur.executemany(INSERT_SQL, market_history_list)
    db_conn.commit()
    print(f"[SUCCESS] Upserted {db_cur.rowcount} records into table '{sql_table}'")
except connector.Error as err:
    db_conn.rollback()
    print(f"[ERROR] Failed to upsert records into table '{sql_table}': {err}")
finally:
    db_cur.close()
    db_conn.close()
    print("[INFO] MySQL connection closed")

[SUCCESS] Upserted 540 records into table 'market_history'
[INFO] MySQL connection closed
